# Serializing Components 

AutoGen provides a  {py:class}`~autogen_core.Component` configuration class that defines behaviours for to serialize/deserialize component into declarative specifications. This is useful for debugging, visualizing, and even for sharing your work with others. In this notebook, we will demonstrate how to serialize multiple components to a declarative specification like a JSON file.  


```{note}
This is work in progress
``` 

We will be implementing declarative support for the following components:

- Termination conditions ✔️
- Tools 
- Agents 
- Teams 


### Termination Condition Example 

In the example below, we will define termination conditions (a part of an agent team) in python, export this to a dictionary/json and also demonstrate how the termination condition object can be loaded from the dictionary/json. 
 

In [1]:
from autogen_agentchat.conditions import MaxMessageTermination, StopMessageTermination

max_termination = MaxMessageTermination(5)
stop_termination = StopMessageTermination()

or_termination = max_termination | stop_termination

or_term_config = or_termination.dump_component()
print("Config: ", or_term_config.model_dump_json())

new_or_termination = or_termination.load_component(or_term_config)

Config:  {"provider":"autogen_agentchat.base.OrTerminationCondition","component_type":"termination","version":1,"component_version":1,"description":null,"config":{"conditions":[{"provider":"autogen_agentchat.conditions.MaxMessageTermination","component_type":"termination","version":1,"component_version":1,"config":{"max_messages":5}},{"provider":"autogen_agentchat.conditions.StopMessageTermination","component_type":"termination","version":1,"component_version":1,"config":{}}]}}


## Agent Example 

In the example below, we will define an agent in python, export this to a dictionary/json and also demonstrate how the agent object can be loaded from the dictionary/json.

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    # tools=[], # serializing tools is not yet supported
    system_message="Use tools to solve tasks.",
)
user_proxy = UserProxyAgent(name="user")

In [3]:
user_proxy_config = user_proxy.dump_component()  # dump component
print(user_proxy_config.model_dump_json())
up_new = user_proxy.load_component(user_proxy_config)  # load component

{"provider":"autogen_agentchat.agents.UserProxyAgent","component_type":"agent","version":1,"component_version":1,"description":null,"config":{"name":"user","description":"A human user"}}


In [4]:
agent_config = agent.dump_component()  # dump component
print(agent_config.model_dump_json())
agent_new = agent.load_component(agent_config)  # load component

{"provider":"autogen_agentchat.agents.AssistantAgent","component_type":"agent","version":1,"component_version":1,"description":null,"config":{"name":"assistant","model_client":{"provider":"autogen_ext.models.openai.OpenAIChatCompletionClient","component_type":"model","version":1,"component_version":1,"config":{"model":"gpt-4o"}},"handoffs":[{"target":"flights_refunder","description":"Handoff to flights_refunder.","name":"transfer_to_flights_refunder","message":"Transferred to flights_refunder, adopting the role of flights_refunder immediately."},{"target":"user","description":"Handoff to user.","name":"transfer_to_user","message":"Transferred to user, adopting the role of user immediately."}],"model_context":{"provider":"autogen_core.model_context.UnboundedChatCompletionContext","component_type":"chat_completion_context","version":1,"component_version":1,"config":{}},"description":"An agent that provides assistance with ability to use tools.","system_message":"Use tools to solve tasks.

A similar approach can be used to serialize the `MultiModalWebSurfer` agent.

```python
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

agent = MultimodalWebSurfer(
    name="web_surfer",
    model_client=model_client,
    headless=False,
)

web_surfer_config = agent.dump_component()  # dump component
print(web_surfer_config.model_dump_json())

```